In [135]:
import pandas as pd
from selenium import webdriver 
from selenium.webdriver.common.keys import Keys 
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException
from bs4 import BeautifulSoup
from itertools import takewhile, chain
from time import sleep
import re
from bs4.element import Tag
from typing import List
from tqdm import tqdm
import requests
import json
import time


In [37]:
tickers = pd.read_csv("sp500.csv")["Symbol"].values

In [137]:
chrome_options=Options()

driver = webdriver.Chrome(f'/Users/rudra/Library/CloudStorage/OneDrive-Personal/Documents/Analytical_Finance_ML/inflation_trading/chromedriver',options=chrome_options) 

/var/folders/m6/h9jtrhyn259fytxp6j9xh9v00000gn/T/ipykernel_19866/2182610177.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(f'/Users/rudra/Library/CloudStorage/OneDrive-Personal/Documents/Analytical_Finance_ML/inflation_trading/chromedriver',options=chrome_options)


In [85]:
tickers = ['AAPL','ABBV','ABT','ACN','AGN','AIG','ALL','AMGN','AMZN','AXP','BA','BAC','BIIB','BK','BLK','BMY','C','CAT','CELG','CL','CMCSA','COF','COP','COST','CSCO','CVS','CVX','DD','DHR','DIS','DOW','DUK','EMC','EMR','EXC','F','FB','FDX','FOX','FOXA','GD','GE','GILD','GM','GOOG','GOOGL','GS','HAL','HD','HON','IBM','INTC','JNJ','JPM','KMI','KO','LLY','LMT','LOW','MA','MCD','MDLZ','MDT','MET','MMM','MO','MON','MRK','MS','MSFT','NEE','NKE','ORCL','OXY','PCLN','PEP','PFE','PG','PM','PYPL','QCOM','RTN','SBUX','SLB','SO','SPG','T','TGT','TWX','TXN','UNH','UNP','UPS','USB','USD','UTX','V','VZ','WBA','WFC']

In [ ]:
tick_dict = {}

In [132]:
tickers = [i for i in tickers if not i in tick_dict]

In [125]:
tickers = tickers[1:]

In [133]:
tickers

['UTX', 'V', 'VZ', 'WBA', 'WFC']

In [138]:
def element_exists(xpath):
    try:
        driver.find_element(By.XPATH, xpath)
    except NoSuchElementException:
        return False
    return True




for ticker in tqdm(tickers):
    try:
        print(ticker)
        driver.get('https://www.sec.gov/edgar/searchedgar/companysearch')

        time.sleep(3)
        if element_exists("/html/body/div[4]/div/div/button/svg"):
            print("FOUND THAT X")
            driver.find_element(By.XPATH, "/html/body/div[4]/div/div/button/svg").click()
            time.sleep(1)
        
        driver.find_element(By.XPATH, "/html/body/div[1]/div/div/div/div[3]/div/div[2]/div[2]/div/div/article/div/div/div[1]/div/div[2]/div/div/div/div[1]/form/div[2]/input").send_keys(ticker)
        time.sleep(1)
        driver.find_element(By.XPATH, "/html/body/div[1]/div/div/div/div[3]/div/div[2]/div[2]/div/div/article/div/div/div[1]/div/div[2]/div/div/div/div[1]/form/div[2]/input").send_keys(u'\ue007')
        time.sleep(5)
        driver.find_element(By.XPATH, "/html/body/main/div[4]/div[2]/div[3]/h5").click()
        time.sleep(1)
        
    
        url = driver.find_element(By.PARTIAL_LINK_TEXT, "10-K: Annual report").get_attribute('href').replace("/ix?doc=","")
        tick_dict[ticker] = url
    
    except (NoSuchElementException, ElementNotInteractableException):
        continue




  0%|          | 0/5 [00:00<?, ?it/s]

UTX


 20%|██        | 1/5 [00:18<01:13, 18.39s/it]

V


 40%|████      | 2/5 [00:32<00:47, 15.93s/it]

...

 60%|██████    | 3/5 [00:43<00:27, 13.83s/it]

WBA


 80%|████████  | 4/5 [00:54<00:12, 12.72s/it]

WFC


100%|██████████| 5/5 [01:05<00:00, 13.18s/it]


In [147]:
pd.DataFrame(tick_dict.items(), columns=["Ticker", "10-K Link"]).to_csv("10k_links.csv", index=False)